# 敵対的生成ネットワーク（GAN、Generative Adversarial Nets）
ここでは、Kerasを用いて、GANの学習を行う。  
GANによって生成された画像は、imagesフォルダに保存される。  

In [1]:
%matplotlib inline


import numpy as np
import matplotlib.pyplot as plt

try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    !mkdir images
    from tensorflow.keras.datasets import mnist
    from tensorflow.keras.layers import Input, Dense, Reshape, Flatten, Dropout
    from tensorflow.keras.layers import BatchNormalization, Activation, ZeroPadding2D
    from tensorflow.keras.layers import LeakyReLU
    from tensorflow.keras.models import Sequential, Model
    from tensorflow.keras.optimizers import Adam
except:
    print('ローカル環境での実行です')
    from keras.datasets import mnist
    from keras.layers import Input, Dense, Reshape, Flatten, Dropout
    from keras.layers import BatchNormalization, Activation, ZeroPadding2D
    from keras.layers import LeakyReLU
    from keras.models import Sequential, Model
    from keras.optimizers import Adam


Google Colab. 上での実行です


### MNISTデータの読み込み

In [2]:
# Load the MNIST dataset
import tensorflow as tf
mnist = tf.keras.datasets.mnist
(X_train, y_train),(X_test, y_test) = mnist.load_data()

# 28*28の画像データを784のベクトルに変換する
X_train = X_train.reshape(-1, 784) / 255
X_test = X_test.reshape(-1, 784) / 255

# N, H,W,Cの配列形状に変換する
X_train = X_train.reshape(-1, 28, 28, 1)

# -1から1の範囲に収める.  generatorの出力にtanhを設定するため
X_train = (X_train * 2) -1

11501568/11490434 [==============================] - 0s 0us/step


### 条件設定

In [3]:
# 画像サイズ
img_rows = 28
img_cols = 28
# 画像のチャンネル数
channels = 1
# 画像の形式
img_shape = (img_rows, img_cols, channels)

### optimizerを設定

In [4]:
optimizer = Adam(0.0002, 0.5)

### Generatorの定義

In [5]:
def build_generator():
    
    # 入力(ノイズが入いる)
    noise_shape = (100,)

    model = Sequential()

    # 全結合層
    model.add(Dense(256, input_shape=noise_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # 全結合層
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # 全結合層
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    
    # 全結合層
    model.add(Dense(np.prod(img_shape), activation='tanh')) # np.prod((1,2,3))=6
    model.add(Reshape(img_shape))

    # ネットワーク構成の表示
    model.summary()

    # 入力レイヤを定義
    noise = Input(shape=noise_shape)
    
    # modelの出力を定義
    img = model(noise)

    # モデルの定義
    # ノイズを入力し, 画像データを出力するモデルを定義
    return Model(inputs=noise, outputs=img) 

### Discriminatorの定義

In [6]:
def build_discriminator():

    model = Sequential()

    # 画像形式配列を1次元配列に変換
    model.add(Flatten(input_shape=img_shape))
    
    # 全結合層
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    
    # 全結合層
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    
    # 全結合層
    model.add(Dense(1, activation='sigmoid'))
    
    # ネットワーク構成の表示
    model.summary()
    
    # 入力レイヤを定義
    img = Input(shape=img_shape)
    
    # modelの出力を定義. probは0~1の値
    prob = model(img)
    
    # モデルの定義
    # 画像データを入力すると, 0~1の値を返すモデルを定義
    return Model(inputs=img, outputs=prob) 

### Discriminatorの構築とコンパイル

In [7]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# ラベルは0or1なので、binary_crossentropyは、画像ごとに2値分類時のクロスエントロピー誤差を算出している

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 256)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-trai

### GeneratorとDiscriminatorを結合したモデルの構築とコンパイル

In [8]:
# Generatorの構築
generator = build_generator()

# Generatorはノイズを入力として画像を生成する
z = Input(shape=(100,))
img_to_d = generator(z)

# 結合したモデルでは、Generatorしか訓練しないので、パラメータを更新しないように設定しておく
discriminator.trainable = False

# モデルの結合: 入力zにはノイズが入る。 discriminator(img_to_d)は識別器での識別結果
combined = Model(z, discriminator(img_to_d))
combined.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
# ラベルは0or1なので、binary_crossentropyは、画像ごとに2値分類時のクロスエントロピー誤差を算出している

# ネットワーク構成の表示
combined.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 256)               25856     
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization (BatchN  (None, 256)              1024      
 ormalization)                                                   
                                                                 
 dense_4 (Dense)             (None, 512)               131584    
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 512)               0         
                                                                 
 batch_normalization_1 (Batc  (None, 512)              2048      
 hNormalization)                                      

### 画像を生成し保存する関数を定義

In [9]:
def sample_images(epoch):
    # 画像の枚数
    r, c = 5, 5
    
    # サンプリング
    noise = np.random.normal(loc=0, scale=1, size=(r * c, 100)) # 画像1枚につき、100点をサンプリングする
    
    # 画像の生成(gen_imgsは-1~1の値)
    gen_imgs = generator.predict(noise)

    # 0から1の範囲に収める
    gen_imgs = 0.5 * gen_imgs + 0.5

    # 5*5枚の画像を並べて1枚の画像にする
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
            
    # 画像を出力
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()

### 訓練
訓練しながら、生成された画像を一定間隔で保存する。

In [ ]:
epochs = 10000
batch_size = 32
sample_interval = 200 # 画像保存間隔
half_batch_size = int(batch_size / 2) # Discriminatorの学習では、バッチサイズの半分を正解データにし残り半分を偽物データにする


for epoch in range(epochs):

    # ランダムにhalf_batch_sizeの分だけ画像を選択
    idx = np.random.randint(0, X_train.shape[0], half_batch_size)
    imgs = X_train[idx]

    # ノイズをサンプリング
    noise = np.random.normal(0, 1, (half_batch_size, 100))

    # half_batch_sizeの分だけ画像を生成
    gen_imgs = generator.predict(noise)

    # 正解データが半分、偽物データが半分のデータセットをつくる
    concat_data = np.concatenate([imgs, gen_imgs], axis=0)
    concat_label = np.concatenate([np.ones((half_batch_size, 1)), np.zeros((half_batch_size, 1))], axis=0)
    
    # Discriminatorを訓練    
    d_loss, d_acc = discriminator.train_on_batch(concat_data, concat_label)

    
    #  Generatorを訓練するのに必要なノイズを生成する
    noise = np.random.normal(loc=0, scale=1, size=(batch_size, 100))

    # ラベル=1を設定する. GeneratorはDiscriminatorに生成した画像を本物と判定させたい
    label = np.ones((batch_size, 1))

    # 結合したモデルを使ってGeneratorを訓練する
    g_loss, g_acc = combined.train_on_batch(noise, label) # ノイズ, ラベル=1

    # 進捗を表示
    print ("%d [d_loss: %f, d_acc:%s] [g_loss: %f, g_acc:%s]" % (epoch, d_loss, d_acc, g_loss, g_acc))

    # 一定間隔で画像を保存
    if epoch % sample_interval == 0:
        sample_images(epoch)

### [演習]
* 中間層のノードの数を変更し、その影響を確認してみましょう
* 出力間隔(sample_interval)を小刻みにして、生成される画像の変化の様子を細かく確認してみましょう
* 畳み込み層やプーリング層を取り入れて計算し、生成画像を比較してみましょう